In [1]:
# Author: Ben Cameron, 2022
#
#
# Takes k >= 1 as input and returns a list of 5-tuples that give all non-isomorphic k-vertex-critical C5 clique expansions.
# After accounting for the symmetrices of C5, these correspond exactly to the tuples whose entries sum to 2k-1 and 
# whose adjacent entries (includign wrap-around) all sum to at most k-1 
# (see https://doi.org/10.1016/j.dam.2016.05.018 for proof)
#
# Among other things, this is of interest as the all k-vertex-critical (gem, co-gem)-free graphs are complete or C5 clique expansions 
#
def all_crit_tuples(k):
    n = 5
    T = Tuples([i for i in range(1,k-1)],n).list() #list of all n-tuples that will be pruned to valid ones only. Note can only do cliques up to order k-2, otherwise, since no isolated verts, clique of size k immediately and will not be k-vertex-critical

    # We now remove all tuples whose entries sum to a value larger than 2k-1
    #
    badTups = []
    for t in T:
        sm = 0
        for i in range(5):
            sm += t[i]
        #print(sm)
        if sm != 2*k-1:
            badTups.append(t)#creates list of bad tuples for removal after loop
    for b in badTups:
        T.remove(b)
        
    # We now remove all that have clique number >= k
    #
    badTups = []
    for t in T:
        for i in range(5):
            if t[i]+t[(i+1)%5] >= k:
                badTups.append(t)#creates list of bad tuples for removal after loop
                break
    for b in badTups:
        T.remove(b)   
    #
    # We now remove all automorphisms of C5
    badTups = []
    Gr = [[1,2,3,4,0],[2,3,4,0,1],[3,4,0,1,2],[4,0,1,2,3],[0,4,3,2,1],[4,3,2,1,0],[3,2,1,0,4],[2,1,0,4,3],[1,0,4,3,2]]
    for j in range(len(T)):
        C1 = T[j]
        for h in range(j+1,len(T)):
            C2 = T[h]
            if (C2 in badTups) == False:
                for i in range(9):
                    p = Gr[i]
                    flag = True #assume isomorphic, flag otherwise
                    for o in range(n):
                        if C1[o] != C2[p[o]]:
                            flag = False
                            break
                    if flag:
                        badTups.append(C2)
                        break
    for l in badTups:
        T.remove(l)
    return(T)

In [2]:
#
# returns the clique expansion of the graph G on vertex set [1,...,n] by replacing vertex V[i] of G with a clique 
# of order L[i] (len(L) must equal order of G and each entry a positive integer)
#
def clique_expansion(G,L): 
    Gverts = G.vertices()
    n = len(Gverts)
    if len(L) != n:
        return("Error: list not the same size as order of G")
    Gedges = G.edges()
    V = [[] for i in range(n)] #vertex set of final graph partitioned into each clique
    E = [] #edge set of final graph
    for i in range(n): # loop fills out
        for j in range(L[i]):
            V[i].append((i+1,j+1))
    for i in range(n): #adds all edges in the cliques
        C = Combinations(V[i], 2).list()
        for c in C:
            E.append({c[0],c[1]})
    for e in Gedges: #adds edges between cliques based on adjaceny of G
        x = e[0]-1
        y = e[1]-1
        for v in V[x]:
            for u in V[y]:
                E.append({u,v})
    return(Graph(E)) 

In [3]:
# 
# generate all graph6 strings of all k-vertex-critical (gem ,co-gem)-free graphs for k<=16
#
C5 = graphs.CycleGraph(5)
for i in range(4,17):
    G = graphs.CompleteGraph(i)
    L = all_crit_tuples(i)
    print("Number of " + str(i) + "-vertex-critical (gem, co-gem)-free graphs: " + str(len(L)+1))
    print(G.graph6_string())    
    for l in L:
        G = clique_expansion(C5,l)
        print(G.graph6_string())
    print("**************************************************************************")

Number of 4-vertex-critical (gem, co-gem)-free graphs: 2
C~
FxK]G
**************************************************************************
Number of 5-vertex-critical (gem, co-gem)-free graphs: 4
D~{
H~CWw^`
H~CWW^b
H~KwW^B
**************************************************************************
Number of 6-vertex-critical (gem, co-gem)-free graphs: 6
E~~w
J~{GW[N@~__
J~{GW[F@~`_
J~{Ww{F@~@_
J~{WwKF@~B_
J~{WwKFw^B_
**************************************************************************
Number of 7-vertex-critical (gem, co-gem)-free graphs: 11
F~~~w
L~~wGKF@wN_^}@
L~~wGKF@wF_^}B
L~~wW[NBwF_^{B
L~~wGKF?wF_^}F
L~~wW[N?wF_^{F
L~~ww{^?wF_^wF
L~~wW[B?wF_^{N
L~~wW[N?wF}B{F
L~~ww{^?wF{BwF
L~~ww{B?wF{FwN
**************************************************************************
Number of 8-vertex-critical (gem, co-gem)-free graphs: 17
G~~~~{
N~~~wCB?wF_^?~?^~_G
N~~~wCB?wF_^?^?^~_W
N~~~wKF@wN_~?^?^~?W
N~~~wCB?wF_N?^?^~_w
N~~~wKF@wN_N?^?^~?w
N~~~w[NBw^_N?^?^}?w
N~~~wKF@wB_N?^?^~@w
N~~~w[NB

Number of 13-vertex-critical (gem, co-gem)-free graphs: 112
L~~~~~~~~~~~~~
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Nw?N{?F~?@~w?N~_?^~~}?@
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Nw?N{?F~?@~w?F~_?^~~}?B
X~~~~~~~~~~~?B?B_@w?^?B{?Nw?^w?^{?N~?B~w?F~_?^~~{?B
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Nw?N{?F~??~w?F~_?^~~}?F
X~~~~~~~~~~~?B?B_@w?^?B{?Nw?^w?^{?N~??~w?F~_?^~~{?F
X~~~~~~~~~~~?F?F_Bw?~?F{?^w?~w?~{?^~??~w?F~_?^~~w?F
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Nw?N{?B~??~w?F~_?^~~}?N
X~~~~~~~~~~~?B?B_@w?^?B{?Nw?^w?^{?B~??~w?F~_?^~~{?N
X~~~~~~~~~~~?F?F_Bw?~?F{?^w?~w?~{?B~??~w?F~_?^~~w?N
X~~~~~~~~~~~?N?N_Fw@~?N{?~w@~w@~{?B~??~w?F~_?^~~o?N
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Nw?F{?B~??~w?F~_?^~~}?^
X~~~~~~~~~~~?B?B_@w?^?B{?Nw?^w?F{?B~??~w?F~_?^~~{?^
X~~~~~~~~~~~?F?F_Bw?~?F{?^w?~w?F{?B~??~w?F~_?^~~w?^
X~~~~~~~~~~~?N?N_Fw@~?N{?~w@~w?F{?B~??~w?F~_?^~~o?^
X~~~~~~~~~~~?^?^_NwB~?^{@~wB~w?F{?B~??~w?F~_?^~~_?^
X~~~~~~~~~~~?@?@_?w?N?@{?Fw?Fw?F{?B~??~w?F~_?^~~}?~
X~~~~~~~~~~~?B?B_@w?^?B{?Nw?Fw?F{?B~??~w?F~_?^~~{?~
X~~~~~~~~~~~?F?F_Bw?~?F{?^w?Fw?F{?B~??~w?

Number of 15-vertex-critical (gem, co-gem)-free graphs: 197
N~~~~~~~~~~~~~~~~~w
\~~~~~~~~~~~~~~~_?G?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??^~_?N~w?@~~~~_?C
\~~~~~~~~~~~~~~~_?G?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??^~_?F~w?@~~~~_?K
\~~~~~~~~~~~~~~~_?W?F??{?Bw?Fw?F{?B~??~w?F~_?^~??~~??~~_?F~w?@~~~~??K
\~~~~~~~~~~~~~~~_?G?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??N~_?F~w?@~~~~_?[
\~~~~~~~~~~~~~~~_?W?F??{?Bw?Fw?F{?B~??~w?F~_?^~??~~??N~_?F~w?@~~~~??[
\~~~~~~~~~~~~~~~_?w?N?@{?Fw?Nw?N{?F~?@~w?N~_?~~?@~~??N~_?F~w?@~~~}??[
\~~~~~~~~~~~~~~~_?G?B??[?@w?Bw?B{?@~??^w?B~_?N~??N~??N~_?F~w?@~~~~_?{
\~~~~~~~~~~~~~~~_?W?F??{?Bw?Fw?F{?B~??~w?F~_?^~??N~??N~_?F~w?@~~~~??{
\~~~~~~~~~~~~~~~_?w?N?@{?Fw?Nw?N{?F~?@~w?N~_?~~??N~??N~_?F~w?@~~~}??{
\~~~~~~~~~~~~~~~_@w?^?B{?Nw?^w?^{?N~?B~w?^~_@~~??N~??N~_?F~w?@~~~{??{
\~~~~~~~~~~~~~~~_?G?B??[?@w?Bw?B{?@~??^w?B~_?F~??N~??N~_?F~w?@~~~~_@{
\~~~~~~~~~~~~~~~_?W?F??{?Bw?Fw?F{?B~??~w?F~_?F~??N~??N~_?F~w?@~~~~?@{
\~~~~~~~~~~~~~~~_?w?N?@{?Fw?Nw?N{?F~?@~w?N~_?F~??N~??N~_?F~w?@~~~}?@{
\~~~~~~~~~

Number of 16-vertex-critical (gem, co-gem)-free graphs: 253
O~~~~~~~~~~~~~~~~~~~~
^~~~~~~~~~~~~~~~~~w?@??K??w?@w?@{??~??Nw?@~_?F~??N~??N~_?F~w?@~~??N~{??^~~~}??G
^~~~~~~~~~~~~~~~~~w?@??K??w?@w?@{??~??Nw?@~_?F~??N~??N~_?F~w?@~~??F~{??^~~~}??W
^~~~~~~~~~~~~~~~~~w?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??^~_?N~w?B~~??F~{??^~~~{??W
^~~~~~~~~~~~~~~~~~w?@??K??w?@w?@{??~??Nw?@~_?F~??N~??N~_?F~w??~~??F~{??^~~~}??w
^~~~~~~~~~~~~~~~~~w?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??^~_?N~w??~~??F~{??^~~~{??w
^~~~~~~~~~~~~~~~~~w?F??{?Bw?Fw?F{?B~??~w?F~_?^~??~~??~~_?^~w??~~??F~{??^~~~w??w
^~~~~~~~~~~~~~~~~~w?@??K??w?@w?@{??~??Nw?@~_?F~??N~??N~_?B~w??~~??F~{??^~~~}?@w
^~~~~~~~~~~~~~~~~~w?B??[?@w?Bw?B{?@~??^w?B~_?N~??^~??^~_?B~w??~~??F~{??^~~~{?@w
^~~~~~~~~~~~~~~~~~w?F??{?Bw?Fw?F{?B~??~w?F~_?^~??~~??~~_?B~w??~~??F~{??^~~~w?@w
^~~~~~~~~~~~~~~~~~w?N?@{?Fw?Nw?N{?F~?@~w?N~_?~~?@~~?@~~_?B~w??~~??F~{??^~~~o?@w
^~~~~~~~~~~~~~~~~~w?@??K??w?@w?@{??~??Nw?@~_?F~??N~??F~_?B~w??~~??F~{??^~~~}?Bw
^~~~~~~~~~~~~~~~~~w?B??[?@w?Bw?B{?@~??